In [1]:
# Import dependencies.

import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
# Import the API key.
from config import g_key

In [2]:
# Import the WeatherPy_vacation.csv file from Part 1 as a new DataFrame.

city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Kapaa,US,2020-04-20 06:47:07,22.08,-159.32,75.2,69,75,2.28,broken clouds,0.0,0.0
1,1,Longyearbyen,SJ,2020-04-20 06:42:31,78.22,15.64,21.2,79,75,20.80,shower rain,0.0,0.0
2,2,Cape Town,ZA,2020-04-20 06:46:00,-33.93,18.42,63.0,77,75,16.11,broken clouds,0.0,0.0
3,3,Puerto Ayora,EC,2020-04-20 06:47:08,-0.74,-90.35,79.0,87,65,5.99,broken clouds,0.0,0.0
4,4,Show Low,US,2020-04-20 06:47:08,34.25,-110.03,43.0,52,1,9.17,clear sky,0.0,0.0


In [3]:
# Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not using conditional statements.

# Configure gmaps to use the Google API key.
gmaps.configure(api_key = g_key)

# Ask the customer for a preferred minimum and maximum temperature value.
# Ask the customer whether they prefer it to be raining or snowing.

min_temp = float(input("What is the minimum temperature you would prefer during your trip? "))
max_temp = float(input("What is the maximum temperature you would prefer during your trip? "))
raining = (input("Would you prefer it to be raining? (yes/no) "))
snowing = (input("Would you prefer it to be snowing? (yes/no) "))

What is the minimum temperature you would prefer during your trip? 60
What is the maximum temperature you would prefer during your trip? 80
Would you prefer it to be raining? (yes/no) no
Would you prefer it to be snowing? (yes/no) no


In [4]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Kapaa,US,2020-04-20 06:47:07,22.08,-159.32,75.20,69,75,2.28,broken clouds,0.000000,0.0
2,2,Cape Town,ZA,2020-04-20 06:46:00,-33.93,18.42,63.00,77,75,16.11,broken clouds,0.000000,0.0
3,3,Puerto Ayora,EC,2020-04-20 06:47:08,-0.74,-90.35,79.00,87,65,5.99,broken clouds,0.000000,0.0
6,6,Rikitea,PF,2020-04-20 06:47:08,-23.12,-134.97,76.80,71,61,6.62,broken clouds,0.000000,0.0
7,7,Busselton,AU,2020-04-20 06:47:08,-33.65,115.33,66.99,73,97,8.99,light rain,0.011811,0.0
8,8,San Cristobal,VE,2020-04-20 06:47:09,7.77,-72.22,66.31,63,84,1.34,light rain,0.024016,0.0
9,9,Sao Filipe,CV,2020-04-20 06:47:09,14.90,-24.50,70.47,75,0,15.88,clear sky,0.000000,0.0
10,10,Esperance,AU,2020-04-20 06:47:09,-33.87,121.90,69.80,43,14,17.22,few clouds,0.000000,0.0
16,16,Bambous Virieux,MU,2020-04-20 06:47:11,-20.34,57.76,77.00,83,75,14.99,light intensity shower rain,0.000000,0.0
17,17,Imbituba,BR,2020-04-20 06:47:11,-28.24,-48.67,62.60,100,1,5.82,clear sky,0.000000,0.0


In [5]:
# Create DataFrame to store hotel names, city, country, max temp, and coordinates.

hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,75.20,broken clouds,22.08,-159.32,
2,Cape Town,ZA,63.00,broken clouds,-33.93,18.42,
3,Puerto Ayora,EC,79.00,broken clouds,-0.74,-90.35,
6,Rikitea,PF,76.80,broken clouds,-23.12,-134.97,
7,Busselton,AU,66.99,light rain,-33.65,115.33,
8,San Cristobal,VE,66.31,light rain,7.77,-72.22,
9,Sao Filipe,CV,70.47,clear sky,14.90,-24.50,
10,Esperance,AU,69.80,few clouds,-33.87,121.90,
16,Bambous Virieux,MU,77.00,light intensity shower rain,-20.34,57.76,
17,Imbituba,BR,62.60,clear sky,-28.24,-48.67,


In [6]:
# Set hotel search parameter.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [7]:
# Loop through the data.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,75.20,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
2,Cape Town,ZA,63.00,broken clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
3,Puerto Ayora,EC,79.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
6,Rikitea,PF,76.80,broken clouds,-23.12,-134.97,Pension Maro'i
7,Busselton,AU,66.99,light rain,-33.65,115.33,Observatory Guest House
8,San Cristobal,VE,66.31,light rain,7.77,-72.22,Pirineos
9,Sao Filipe,CV,70.47,clear sky,14.90,-24.50,Tortuga B&B
10,Esperance,AU,69.80,few clouds,-33.87,121.90,Hospitality Esperance
16,Bambous Virieux,MU,77.00,light intensity shower rain,-20.34,57.76,Casa Tia Villa
17,Imbituba,BR,62.60,clear sky,-28.24,-48.67,Praia Hotel Imbituba


In [9]:
# Save and upload the new DataFrame as WeatherPy_vacation.csv.
output_data_file = "weather_data/WeatherPy_vacation.csv"

# Export the data into the csv file.
hotel_df.to_csv(output_data_file)

In [10]:
# Add a heatmap of temperature for the vacation spots.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Add a heatmap of temperature for the vacation spots and marker for each city.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [14]:
# Store the data.

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))